In [ ]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"xinkaichen97","key":"a31b1965f229c3f21b0282be2d9cdd3f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config path -p /content

In [ ]:
!pip install ipdb
import os, zipfile, tarfile, ipdb
os.environ['KAGGLE_USERNAME'] = "xinkaichen97" 
os.environ['KAGGLE_KEY'] = "a31b1965f229c3f21b0282be2d9cdd3f"

In [ ]:
!kaggle competitions download -c 11-785-fall-20-homework-1-part-2

In [ ]:
import zipfile
with zipfile.ZipFile("train.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("train_labels.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("dev.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("dev_labels.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("test.npy.zip","r") as z:
  z.extractall(".")
with zipfile.ZipFile("sample.csv.zip","r") as z:
  z.extractall(".")

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
class CorrectDataset(Dataset):
    def __init__(self, features_file, labels_file, context_size):
        self.inputs = np.load(features_file, allow_pickle=True)
        self.indices_dict = self.get_indices()
        self.pad_utterances(context_size)
        print(self.inputs[0].shape)
        self.labels = np.load(labels_file, allow_pickle=True)
        print(self.labels.shape)
        print(len(self.indices_dict.keys()))
        self.context_size = context_size
    
    def __len__(self):
        return len(self.indices_dict.keys())
  
    def __getitem__(self, idx):
        utter_idx, within_utter_idx = self.indices_dict[idx]
        within_utter_idx_pad = within_utter_idx + self.context_size
        return torch.flatten(self.inputs[utter_idx][within_utter_idx_pad - self.context_size: within_utter_idx_pad + self.context_size + 1, :]), self.labels[utter_idx][within_utter_idx]

    def get_indices(self):
        total = 0
        indices_dict = {}
        for i in range(len(self.inputs)):
            utterance = self.inputs[i]
            for j in range(len(utterance)):
                indices_dict[total + j] = (i, j)
            total += len(utterance)
        print(total)
        return indices_dict
    
    def pad_utterances(self, context_size):
        for i in range(len(self.inputs)):
            self.inputs[i] = F.pad(input=torch.Tensor(self.inputs[i]), pad=(0,0,context_size,context_size), mode='constant', value=0)
    
    def get_context(self, utterance, i, context_size):
        return torch.flatten(utterance[i - context_size: i + context_size + 1, :])

In [ ]:
train_dataset = CorrectDataset(features_file='train.npy', labels_file='train_labels.npy', context_size=20)

In [ ]:
print(train_dataset[3190][0].shape)
print(len(train_dataset))

In [ ]:
dev_dataset = CorrectDataset(features_file='dev.npy', labels_file='dev_labels.npy', context_size=20)
devloader = DataLoader(dev_dataset, batch_size=5000, num_workers=4)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear((1 + 2 * 20) * 13, 1024)
        self.bn1 = nn.BatchNorm1d(1024)  
        self.fc2 = nn.Linear(1024, 2048)
        self.bn2 = nn.BatchNorm1d(2048) 
        self.fc3 = nn.Linear(2048, 2048)
        self.bn3 = nn.BatchNorm1d(2048)
        self.fc4 = nn.Linear(2048, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.fc5 = nn.Linear(1024, 346)

    def forward(self, x):
        y = self.bn1(F.relu(self.fc1(x)))
        y = self.bn2(F.relu(self.fc2(y)))
        y = self.bn3(F.relu(self.fc3(y)))
        y = self.bn3(F.relu(self.fc3(y)))
        y = self.bn4(F.relu(self.fc4(y)))
        y = self.fc5(y)
        return y

In [ ]:
dataloader = DataLoader(train_dataset, batch_size=5000, shuffle=True, num_workers=4)
model = Net().cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)

In [ ]:
# training
val_accuracies = []
train_losses = []
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
num_epochs = 10
for epoch in range(num_epochs):
    if epoch % 2 == 1:
      scheduler.step()  
    for i, (batch_data, batch_labels) in enumerate(dataloader):
        batch_labels = batch_labels.cuda()
        batch_data = batch_data.cuda()
        running_loss = 0.0
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            train_losses.append(running_loss)
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    val_accuracies.append(validate(devloader))
print('Finished Training')

In [ ]:
# validation
def validate(devloader):
    num_correct = 0
    for i, (batch_data, batch_labels) in enumerate(devloader):
            batch_labels = batch_labels.cuda()
            batch_data = batch_data.cuda()
            outputs = model(batch_data)
            pred = torch.argmax(outputs, dim=1)
            current_correct = len([pred[i] for i in range(len(pred)) if pred[i] == batch_labels[i]])
            num_correct += current_correct
    accuracy = num_correct / len(dev_dataset)
return accuracy

In [ ]:
class TestDataset(Dataset):
    def __init__(self, features_file, context_size):
        self.inputs = np.load(features_file, allow_pickle=True)
        self.indices_dict = self.get_indices()
        self.pad_utterances(context_size)
        self.context_size = context_size
    
    def __len__(self):
        return len(self.indices_dict.keys())
  
    def __getitem__(self, idx):
        utter_idx, within_utter_idx = self.indices_dict[idx]
        within_utter_idx_pad = within_utter_idx + self.context_size
        return torch.flatten(self.inputs[utter_idx][within_utter_idx_pad - self.context_size: within_utter_idx_pad + self.context_size + 1, :])
    
    def get_indices(self):
        total = 0
        indices_dict = {}
        for i in range(len(self.inputs)):
            utterance = self.inputs[i]
            for j in range(len(utterance)):
                indices_dict[total + j] = (i, j)
            total += len(utterance)
        print(total)
        return indices_dict
    
    def pad_utterances(self, context_size):
        for i in range(len(self.inputs)):
            self.inputs[i] = F.pad(input=torch.Tensor(self.inputs[i]), pad=(0,0,context_size,context_size), mode='constant', value=0)
    
    def get_context(self, utterance, i, context_size):
        return torch.flatten(utterance[i - context_size: i + context_size + 1, :])

In [ ]:
torch.save(model.state_dict(), "model_20_bigger.pt")

In [ ]:
test_set = TestDataset(features_file='test.npy', context_size=20)

1593223
torch.Size([365, 13])
1593223


In [ ]:
testloader = DataLoader(test_set, batch_size=5000)

In [ ]:
# get test predictions
total_preds = np.array([])
for i, batch_data in enumerate(testloader):
  batch_data = batch_data.cuda()
  outputs = model(batch_data)
  pred = np.argmax(outputs.cpu().detach().numpy(), axis=1)
  total_preds = np.append(total_preds, pred)

In [ ]:
# generate submission.csv
with open('submission.csv', 'w') as f:
  f.write('id,label\n')
  for i, label in enumerate(total_preds):
    f.write(str(i) + ',' + str(int(label)) + '\n')

In [ ]:
!kaggle competitions submit -c 11-785-fall-20-homework-1-part-2 -f submission.csv -m ""